In [5]:
from fastai.vision.all import *
import os
import zipfile

# Define the path to the zip file
zip_path = Path("/content/archive (2).zip")

# Extract the zip file contents to a temporary directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/tmp/Brain_Data_Organised')

# Define the path to the extracted dataset
path = Path("/tmp/Brain_Data_Organised/Brain_Data_Organised")  # Adjust if the internal folder structure is different

# Ensure the path is correct and contains image files in subfolders representing classes.
print(path.ls())

# Check if there are any non-image files in your data directory
# and change get_items to only select image files.
def get_image_files_filtered(path):
    return [f for f in get_image_files(path) if os.path.splitext(f)[1].lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']]

# Load data using FastAI's DataBlock with augmentations and weighted sampling
stroke_dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files_filtered,  # Use the filtered function here
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(224)],
    batch_tfms=[*aug_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_warp=0.2), Normalize.from_stats(*imagenet_stats)]
)

dls = stroke_dblock.dataloaders(path, bs=32)

# Compute class weights to handle imbalance
from collections import Counter

y_counts = Counter([parent_label(f) for f in get_image_files(path)])
class_weights = torch.tensor([1/y_counts['Normal'], 1/y_counts['Stroke']])

# List of models to train in parallel
models = [efficientnet_b0, efficientnet_b3, resnet34, resnet50, densenet121, alexnet]
model_names = ["EfficientNet-B0", "EfficientNet-B3", "ResNet-34", "ResNet-50", "DenseNet-121", "AlexNet"]
accuracies = {}

for model, name in zip(models, model_names):
    print(f"Training model: {name}")
    learn = cnn_learner(dls, model, metrics=accuracy, loss_func=CrossEntropyLossFlat(weight=class_weights))
    learn.fine_tune(10, freeze_epochs=3)
    final_accuracy = learn.validate()[1]
    accuracies[name] = final_accuracy  # Remove .item()
    print(f"{name} Final Accuracy: {final_accuracy:.4f}")
    learn.export(f"stroke_classifier_{name}.pkl")

# Print all model accuracies
print("Model Accuracies:")
for name, acc in accuracies.items():
    print(f"{name}: {acc:.4f}")


[Path('/tmp/Brain_Data_Organised/Brain_Data_Organised/Normal'), Path('/tmp/Brain_Data_Organised/Brain_Data_Organised/Stroke')]
Training model: EfficientNet-B0


/usr/local/lib/python3.11/dist-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,time
0,1.168209,1.065331,0.514000,00:19
1,1.181904,0.881228,0.654000,00:20
2,0.978662,0.725104,0.636000,00:19


epoch,train_loss,valid_loss,accuracy,time
0,0.745404,0.581595,0.690000,00:21
1,0.700458,0.633665,0.656000,00:21
2,0.683388,0.599051,0.726000,00:20
3,0.660010,0.572525,0.728000,00:22
4,0.609374,0.533394,0.742000,00:22
5,0.601189,0.537892,0.742000,00:21
6,0.575612,0.521690,0.764000,00:22
7,0.553513,0.510311,0.786000,00:22
8,0.549582,0.510989,0.766000,00:21
9,0.529575,0.518069,0.772000,00:21


EfficientNet-B0 Final Accuracy: 0.7720
Training model: EfficientNet-B3


...

DenseNet-121 Final Accuracy: 0.9160
Training model: AlexNet


/usr/local/lib/python3.11/dist-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 169MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,1.210930,0.962212,0.594000,00:18
1,1.085284,0.814457,0.618000,00:17
2,0.954164,0.683846,0.682000,00:17


epoch,train_loss,valid_loss,accuracy,time
0,0.728594,0.600659,0.698000,00:19
1,0.702638,0.591476,0.722000,00:17
2,0.652767,0.519537,0.770000,00:19
3,0.626102,0.452860,0.790000,00:17
4,0.594402,0.446070,0.792000,00:18
5,0.559523,0.469826,0.804000,00:18
6,0.525371,0.396710,0.828000,00:17
7,0.503512,0.381910,0.800000,00:19
8,0.481095,0.365637,0.832000,00:17
9,0.449823,0.366085,0.844000,00:18


AlexNet Final Accuracy: 0.8440
Model Accuracies:
EfficientNet-B0: 0.7720
EfficientNet-B3: 0.7320
ResNet-34: 0.9060
ResNet-50: 0.8680
DenseNet-121: 0.9160
AlexNet: 0.8440
